In [1]:
%load_ext autoreload
%load_ext autotime

time: 237 µs (started: 2023-12-10 19:56:02 -08:00)


In [2]:
import os
import sys
wisio_dir = os.path.abspath("../")
sys.path.append(wisio_dir)
import warnings
warnings.filterwarnings('ignore')

time: 1.58 ms (started: 2023-12-10 19:56:02 -08:00)


In [3]:
import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd
print('pandas', pd.__version__)
print('dask', dask.__version__)
print('numpy', np.__version__)

pandas 2.1.2
dask 2023.9.3
numpy 1.24.3
time: 1.43 s (started: 2023-12-10 19:56:02 -08:00)


In [4]:
# trace_path = '/usr/workspace/iopp/wisio_logs/darshan_dlp_overhead_1_40/haridev_overhead_id3822049-3822049_11-7-66326-7292532907421145426_1.darshan'
# trace_path = '/usr/workspace/iopp/wisio_logs/darshan_dlp_overhead_4_40/haridev_overhead_id1873753-1873753_11-7-66866-5106999951320747098_1.darshan'
trace_path = '/usr/workspace/iopp/wisio_logs/darshan_dlp_overhead_8_40/haridev_overhead_id1875427-1875427_11-7-66903-15281823531808078074_2.darshan'

time: 907 µs (started: 2023-12-10 19:56:04 -08:00)


In [5]:
%autoreload 2

from wisio.cluster_management import load_cluster_config
from wisio.darshan import DarshanAnalyzer

cluster_config = load_cluster_config('cluster_config_local.yml')
cluster_config.local_dir = '/var/tmp/wisio-darshan-dlp'

analyzer = DarshanAnalyzer(
    checkpoint=True,
    checkpoint_dir='/usr/workspace/iopp/wisio_logs/_checkpoints/darshan_dlp_overhead_8_40',
    cluster_config=cluster_config,
    working_dir='.wisio/darshan_dlp_overhead_8_40',
    debug=True,
)

analyzer

[INFO] [19:56:05] Initializing Darshan analyzer [/g/g91/izzet/projects/wisio/wisio/analyzer.py:88]
[DEBUG] [19:56:05] Using selector: EpollSelector [/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9/asyncio/selector_events.py:54]


time: 3.59 s (started: 2023-12-10 19:56:04 -08:00)


In [6]:
analyzer.cluster_manager.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44763/status,
Dashboard: http://127.0.0.1:44763/status,Workers: 8
Total threads: 48,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46313,Workers: 8
Dashboard: http://127.0.0.1:44763/status,Total threads: 48
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:45879,Total threads: 6
Dashboard: http://127.0.0.1:41741/status,Memory: 0 B
Nanny: tcp://127.0.0.1:33365,


time: 57.4 ms (started: 2023-12-10 19:56:09 -08:00)


In [8]:
%autoreload 2

res = analyzer.analyze_dxt(
    trace_path_pattern=trace_path, 
    metrics=['time'],
    view_types=['proc_name'],
)

[DEBUG] [19:57:14]  Refreshing name_records for mod=POSIX [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [19:57:14]  Refreshing name_records for mod=MPI-IO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [19:57:14]  Refreshing name_records for mod=STDIO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [19:57:14]  Refreshing name_records for mod=DXT_POSIX [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [19:57:14]  Refreshing name_records for mod=DXT_MPIIO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [19:57:14]  Refreshing name_records for mod=LUSTRE [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[INFO] [19:57:19] Read traces (5.460269350558519) [/g/g91/izzet/projects/wisio/wisio/darsh

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│  I/O Time             0.32 seconds                                                                              │
│                       ├── Read - 0.30 seconds (92.27%)                                                          │
│                       ├── Write - 0.04 seconds (11.75%)                                                         │
│                       └── Metadata - 0.00 seconds (0.00%)                                                       │
│  I/O Ops              320,320 ops                                                                               │
│                       ├── Read - 320 ops (0.10%)                                                                │
│                       ├── Write - 320,000 ops (99.90%)                                                          │
│                       └── Metadata - 0 ops (0.00%)                                                              │
│  I/O Size             39.18 GB                                                                                  │
│                       ├── Read - 0.12 GB (0.31%)                                                                │
│                       └── Write - 39.06 GB (99.69%)                                                             │
│  Read Xfer            1 MB - 320 ops                                                                            │
│                       └── 256 KB-1 MB - 320 ops (100.00%)                                                       │
│  Write Xfer           256 KB - 18,239 ops                                                                       │
│                       └── 64-256 KB - 18,239 ops (100.00%)                                                      │
│  Node(s)              1 node(s)                                                                                 │
│                       └── localhost - 0.32 s (100.00%) - 0.12/39.06 GB R/W (0.31/99.69%) - 320,320 ops          │
│                           (100.00%)                                                                             │
│  App(s)               1 app(s)                                                                                  │
│                       └── app - 0.32 s (100.00%) - 0.12/39.06 GB R/W (0.31/99.69%) - 320,320 ops (100.00%)      │
│  Process(es)/Rank(s)  320 process(es)/rank(s)                                                                   │
│  Files                320 files                                                                                 │
│                       ├── Shared: 1 files (0.31%)                                                               │
│                       └── FPP: 319 files (99.69%)                                                               │
│  Access Pattern       100.00% Sequential - 0.00% Random                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭──────────────────────────────────────────────── I/O Bottlenecks ────────────────────────────────────────────────╮
│  small_reads (204 bottlenecks)                                                                                  │
│  ├── proc_name (101 bottlenecks)                                                                                │
│  │   ├── 1 process accesses 2 files within 3 time periods and has an I/O time of 0.32 seconds which is 100.00%  │
│  │   │   of overall I/O time of the workload.                                                                   │
│  │   │   ├── 'read' time is 92.27% (0.30 seconds) of I/O time.                                                  │
│  │   │   └── Average 'read's are 0.40 KB, which is smaller than 256 KB.                                         │
│  │   ├── 1 process accesses 2 files within 3 time peri

time: 14.3 s (started: 2023-12-10 19:57:14 -08:00)


In [37]:
summ = 0
for metric in res.evaluated_views:
    for view_key in res.evaluated_views[metric]:
        # print(metric, view_key, len(res.evaluated_views[metric][view_key].bottlenecks))
        summ = summ + len(res.evaluated_views[metric][view_key].bottlenecks)
summ 

/g/g91/izzet/projects/wisio/wisio/analysis.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['bw_intensity'] = ddf['bw'] * ddf['intensity']
/g/g91/izzet/projects/wisio/wisio/analysis.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['att_perf'] = ddf[['iops', 'bw_intensity']].min(axis=1)
/g/g91/izzet/projects/wisio/wisio/analysis.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

253

time: 4.8 s (started: 2023-11-29 17:00:54 -08:00)


In [26]:
%autoreload 2

res = analyzer.analyze_dxt(
    trace_path_pattern=trace_path, 
    metrics=['time'],
    # view_types=['proc_name'],
)

[DEBUG] [16:46:28]  Refreshing name_records for mod=POSIX [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:46:28]  Refreshing name_records for mod=MPI-IO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:46:28]  Refreshing name_records for mod=STDIO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:46:28]  Refreshing name_records for mod=DXT_POSIX [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:46:28]  Refreshing name_records for mod=DXT_MPIIO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:46:28]  Refreshing name_records for mod=LUSTRE [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:46:31] Read traces (2.7298541786149144) [/g/g91/izzet/projects/wisio/wisio/uti

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│  I/O Time        0.17 seconds                                                                                   │
│                  ├── Read - 0.14 seconds (82.93%)                                                               │
│                  ├── Write - 0.03 seconds (18.36%)                                                              │
│                  └── Metadata - 0.00 seconds (0.00%)                                                            │
│  I/O Ops         160,160 ops                                                                                    │
│                  ├── Read - 160 ops (0.10%)                                                                     │
│                  ├── Write - 160,000 ops (99.90%)                                                               │
│                  └── Metadata - 0 ops (0.00%)                                                                   │
│  I/O Size        19.65 GB                                                                                       │
│                  ├── Read - 0.12 GB (0.62%)                                                                     │
│                  └── Write - 19.53 GB (99.38%)                                                                  │
│  Read Xfer       1 MB - 160 ops                                                                                 │
│                  └── 256 KB-1 MB - 160 ops (100.00%)                                                            │
│  Write Xfer      256 KB - 8,639 ops                                                                             │
│                  └── 64-256 KB - 8,639 ops (100.00%)                                                            │
│  Node(s)         1 node(s)                                                                                      │
│                  └── localhost - 0.17 s (100.00%) - 0.12/19.53 GB R/W (0.62/99.38%) - 160,160 ops (100.00%)     │
│  App(s)          1 app(s)                                                                                       │
│                  └── app - 0.17 s (100.00%) - 0.12/19.53 GB R/W (0.62/99.38%) - 160,160 ops (100.00%)           │
│  Files           160 files                                                                                      │
│                  ├── Shared: 1 files (0.62%)                                                                    │
│                  └── FPP: 159 files (99.38%)                                                                    │
│  Access Pattern  100.00% Sequential - 0.00% Random                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭──────────────────────────────────────────────── I/O Bottlenecks ────────────────────────────────────────────────╮
│  small_reads                                                                                                    │
│  ├── file_name                                                                                                  │
│  │   └── 160 processes access 1 file within 3 time periods and have an I/O time of 0.14 seconds which is        │
│  │       82.93% of overall I/O time of the workload.                                                            │
│  ├── proc_name                                                                                                  │
│  │   └── 50 processes access 51 files within 28 time periods and have an I/O time of 0.14 seconds which is      │
│  │       79.99% of overall I/O time of the workload.                                                            │
│  ├── file_name > proc_name                                                                                      │
│  │   └── 1 process accesses 2 files within 3 time peri

time: 20.4 s (started: 2023-11-29 16:46:28 -08:00)


In [12]:
import darshan 

rep = darshan.DarshanReport(trace_path, read_all=True)
rep 

[DEBUG] [16:32:07]  Refreshing name_records for mod=POSIX [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:32:07]  Refreshing name_records for mod=MPI-IO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:32:07]  Refreshing name_records for mod=STDIO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]


[DEBUG] [16:32:07]  Refreshing name_records for mod=DXT_POSIX [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:32:07]  Refreshing name_records for mod=DXT_MPIIO [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]
[DEBUG] [16:32:07]  Refreshing name_records for mod=LUSTRE [/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/darshan/report.py:519]


time: 251 ms (started: 2023-11-29 16:32:07 -08:00)


In [13]:
df = rep.records['DXT_POSIX'].to_df()
df

[{'id': 6335191231442812216,
  'rank': 0,
  'hostname': 'corona239',
  'write_count': 1,
  'read_count': 1000,
  'write_segments':    offset  length  start_time  end_time
  0       0  819200    0.582285  0.656295,
  'read_segments':         offset  length  start_time  end_time
  0            0  131072    3.358670  3.358777
  1       131072  131072    3.477599  3.477646
  2       262144  131072    3.477687  3.477726
  3       393216  131072    3.477766  3.477803
  4       524288  131072    3.477847  3.477882
  ..         ...     ...         ...       ...
  995  130416640  131072    3.529494  3.529533
  996  130547712  131072    3.529545  3.529586
  997  130678784  131072    3.529589  3.529632
  998  130809856  131072    3.529639  3.529683
  999  130940928  131072    3.529686  3.529721
  
  [1000 rows x 4 columns]},
 {'id': 6335191231442812216,
  'rank': 1,
  'hostname': 'corona239',
  'write_count': 1,
  'read_count': 0,
  'write_segments':    offset  length  start_time  end_time
  0  8

time: 2.91 s (started: 2023-11-29 16:32:08 -08:00)


In [18]:
len(df)

319

time: 2.9 ms (started: 2023-11-29 16:04:53 -08:00)


In [20]:
time_granularity = 1e6
time_granularity

1000000.0

time: 3.02 ms (started: 2023-11-29 16:35:17 -08:00)


In [23]:
time_granularity = 1e3

dxt_posix = pd.DataFrame(df)

read_id = []
read_rank = []
read_length = []
read_offsets = []
read_end_time = []
read_start_time = []
read_operation = []
read_hostname = []
read_io_cat = []

write_id = []
write_rank = []
write_length = []
write_offsets = []
write_end_time = []
write_start_time = []
write_operation = []
write_hostname = []
write_io_cat = []

# start = time.time()
for r in zip(dxt_posix['rank'], dxt_posix['read_segments'], dxt_posix['write_segments'], dxt_posix['id'], dxt_posix['hostname']):
    if not r[1].empty:
        read_id.append([r[3]] * len((r[1]['length'].to_list())))
        read_rank.append([r[0]] * len((r[1]['length'].to_list())))
        read_length.append(r[1]['length'].to_list())
        read_end_time.append(r[1]['end_time'].to_list())
        read_start_time.append(r[1]['start_time'].to_list())
        read_operation.append(['read'] * len((r[1]['length'].to_list())))
        read_offsets.append(r[1]['offset'].to_list())
        read_hostname.append([r[4]] * len((r[1]['length'].to_list())))
        read_io_cat.append([2]*len((r[1]['length'].to_list())))

    if not r[2].empty:
        write_id.append([r[3]] * len((r[2]['length'].to_list())))     
        write_rank.append([r[0]] * len((r[2]['length'].to_list())))
        write_length.append(r[2]['length'].to_list())
        write_end_time.append(r[2]['end_time'].to_list())
        write_start_time.append(r[2]['start_time'].to_list())
        write_operation.append(['write'] * len((r[2]['length'].to_list())))
        write_offsets.append(r[2]['offset'].to_list())
        write_hostname.append([r[4]] * len((r[2]['length'].to_list())))
        write_io_cat.append([1]*len((r[2]['length'].to_list())))

file_name = [rep.data['name_records'][element] for nestedlist in read_id for element in nestedlist]
rank = [element for nestedlist in read_rank for element in nestedlist]
length = [element for nestedlist in read_length for element in nestedlist]
offsets = [element for nestedlist in read_offsets for element in nestedlist]
end_time = [element for nestedlist in read_end_time for element in nestedlist]
operation = [element for nestedlist in read_operation for element in nestedlist]
start_time = [element for nestedlist in read_start_time for element in nestedlist]
time_range = [int(element*time_granularity) for nestedlist in read_start_time for element in nestedlist]
hostname = [element for nestedlist in read_hostname for element in nestedlist]
io_cat = [element for nestedlist in read_io_cat for element in nestedlist]

file_name.extend([rep.data['name_records'][element] for nestedlist in write_id for element in nestedlist])
rank.extend([element for nestedlist in write_rank for element in nestedlist])
length.extend([element for nestedlist in write_length for element in nestedlist])
offsets.extend([element for nestedlist in write_offsets for element in nestedlist])
end_time.extend([element for nestedlist in write_end_time for element in nestedlist])
operation.extend([element for nestedlist in write_operation for element in nestedlist])
start_time.extend([element for nestedlist in write_start_time for element in nestedlist])
time_range.extend([int(element*time_granularity) for nestedlist in write_start_time for element in nestedlist])
hostname.extend([element for nestedlist in write_hostname for element in nestedlist])
io_cat.extend([element for nestedlist in write_io_cat for element in nestedlist])

dxt_posix_df = pd.DataFrame(
    {
    'file_name': file_name,
    'proc_name': rank,
    'size': length,
    'end_time': end_time,
    'start_time': start_time,
    'func_id': operation,
    # 'offsets': offsets,
    'hostname': hostname,
    'io_cat': io_cat,
    'time_range': time_range,
    })

dxt_posix_df['cat'] = 0
dxt_posix_df['acc_pat'] = 0
dxt_posix_df['count'] = 1
dxt_posix_df['time'] = dxt_posix_df['end_time'] - dxt_posix_df['start_time']

dxt_posix_df

,file_name,proc_name,size,end_time,start_time,func_id,hostname,io_cat,time_range,cat,acc_pat,count,time
0,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,0,131072,3.358777,3.358670,read,corona239,2,3358,0,0,1,0.000107
1,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,0,131072,3.477646,3.477599,read,corona239,2,3477,0,0,1,0.000047
2,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,0,131072,3.477726,3.477687,read,corona239,2,3477,0,0,1,0.000039
3,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,0,131072,3.477803,3.477766,read,corona239,2,3477,0,0,1,0.000036
4,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,0,131072,3.477882,3.477847,read,corona239,2,3477,0,0,1,0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160155,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,155,819200,0.655136,0.582232,write,corona242,1,582,0,0,1,0.072903
160156,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,156,819200,0.645535,0.582336,write,corona242,1,582,0,0,1,0.063199
160157,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,157,819200,0.653189,0.582305,write,corona242,1,582,0,0,1,0.070884
160158,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,158,819200,0.648862,0.582312,write,corona242,1,582,0,0,1,0.066551


time: 545 ms (started: 2023-11-29 16:36:11 -08:00)


In [18]:
time_granularity

100000.0

time: 3.11 ms (started: 2023-11-29 16:34:44 -08:00)


In [23]:
dxt_posix_read_data

,id,rank,length,end_time,start_time,operation,offsets
0,6335191231442812216,0,131072,3.358777,3.358670,read,0
1,6335191231442812216,0,131072,3.477646,3.477599,read,131072
2,6335191231442812216,0,131072,3.477726,3.477687,read,262144
3,6335191231442812216,0,131072,3.477803,3.477766,read,393216
4,6335191231442812216,0,131072,3.477882,3.477847,read,524288
...,...,...,...,...,...,...,...
159995,16131063875095799968,159,131072,3.529516,3.529479,read,130416640
159996,16131063875095799968,159,131072,3.529556,3.529526,read,130547712
159997,16131063875095799968,159,131072,3.529606,3.529576,read,130678784
159998,16131063875095799968,159,131072,3.529653,3.529622,read,130809856


time: 29.9 ms (started: 2023-11-29 16:06:24 -08:00)


In [20]:
dff = pd.DataFrame(df)
dff 

,id,rank,hostname,write_count,read_count,write_segments,read_segments
0,6335191231442812216,0,corona239,1,1000,offset length start_time end_time 0 ...,offset length start_time end_time 0...
1,6335191231442812216,1,corona239,1,0,offset length start_time end_time 0 819...,Empty DataFrame Columns: [] Index: []
2,17415967488970617980,1,corona239,0,1000,Empty DataFrame Columns: [] Index: [],offset length start_time end_time 0...
3,6335191231442812216,2,corona239,1,0,offset length start_time end_time 0 16...,Empty DataFrame Columns: [] Index: []
4,8632927880694132813,2,corona239,0,1000,Empty DataFrame Columns: [] Index: [],offset length start_time end_time 0...
...,...,...,...,...,...,...,...
314,9914971967526331231,157,corona242,0,1000,Empty DataFrame Columns: [] Index: [],offset length start_time end_time 0...
315,6335191231442812216,158,corona242,1,0,offset length start_time end_time 0 ...,Empty DataFrame Columns: [] Index: []
316,16935048567515436115,158,corona242,0,1000,Empty DataFrame Columns: [] Index: [],offset length start_time end_time 0...
317,6335191231442812216,159,corona242,1,0,offset length start_time end_time 0 ...,Empty DataFrame Columns: [] Index: []


time: 81.7 ms (started: 2023-11-29 16:05:17 -08:00)


In [33]:
simdesc = cosine_similarity([z[xc].describe().loc['mean']], [t[xc].describe().loc['mean']])
simdesc

array([[0.93372649]])

time: 110 ms (started: 2023-11-21 17:00:01 -08:00)


In [34]:
# Find similar rows
similar_rows = []
for i in range(len(simdesc)):
    for j in range(i + 1, len(simdesc)):
        # if simdesc[i, j] > threshold:
        similar_rows.append((i, j))

# Display similar rows
for i, j in similar_rows:
    print(f"Row {i} and Row {j} have cosine similarity {simdesc[i, j]}")

time: 889 µs (started: 2023-11-21 17:00:01 -08:00)


In [35]:
bot_time_pn_hl = res.bottlenecks['time'][('proc_name',)].high_level_view
bot_time_pn_ml = res.bottlenecks['time'][('proc_name',)].mid_level_view
bot_time_pn_ll = res.bottlenecks['time'][('proc_name',)].low_level_view

time: 663 µs (started: 2023-11-21 17:00:01 -08:00)


In [36]:
bot_time_pn_hl.sort_values('time_pero', ascending=False).head()

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
proc_name,,,,,,,,,,,,,,,,,,,,,
app#corona241#89#0,0.169967,1001,131072,819200,131891200,0.029014,0.140953,0.0,131072000.0,819200.0,...,7.759826e+08,5889.389001,0.000008,1804820281166176736137356515205775550881292589...,1.837910e+06,1.000000,1.000000,0.024191,critical,0.9
app#corona241#118#0,0.166187,1001,131072,819200,131891200,0.029890,0.136297,0.0,131072000.0,819200.0,...,7.936333e+08,6023.350703,0.000008,1866894165685163063851189717222090542982845782...,1.787333e+06,0.977760,0.977760,0.023653,critical,0.9
app#corona242#134#0,0.163221,1001,131072,819200,131891200,0.029183,0.134038,0.0,131072000.0,819200.0,...,8.080546e+08,6132.802662,0.000008,3261215828067764269173308031257655676222747521...,1.820277e+06,0.960310,0.960310,0.023231,critical,0.9
app#corona240#51#0,0.156551,1001,131072,819200,131891200,0.030285,0.126266,0.0,131072000.0,819200.0,...,8.424801e+08,6394.077430,0.000008,3146234945541156382688027267126553478661966506...,1.755197e+06,0.921069,0.921069,0.022282,critical,0.9
app#corona240#62#0,0.154034,1001,131072,819200,131891200,0.021803,0.132231,0.0,131072000.0,819200.0,...,8.562492e+08,6498.579559,0.000008,2712331605500877258841167499571517861188204678...,2.435301e+06,0.906258,0.906258,0.021923,critical,0.9


time: 211 ms (started: 2023-11-21 17:00:01 -08:00)


In [37]:
bot_time_pn_ml.loc['app#corona242#134#0'].sort_values('time_pero', ascending=False).head()

time  count  size_min  size_max     size  \
proc_name           time_range                                                 
app#corona242#134#0 649         0.134038      1    819200    819200   819200   
                    3496        0.000740     19    131072    131072  2490368   
                    3495        0.000713     24    131072    131072  3145728   
                    3503        0.000705     25    131072    131072  3276800   
                    3524        0.000687     23    131072    131072  3014656   

                                read_time  write_time  metadata_time  \
proc_name           time_range                                         
app#corona242#134#0 649          0.000000    0.134038            0.0   
                    3496         0.000740    0.000000            0.0   
                    3495         0.000713    0.000000            0.0   
                    3503         0.000705    0.000000            0.0   
                    3524         0.000687    0.000000            0.0   

                                read_size  write_size  ...            bw  \
proc_name           time_range                         ...                 
app#corona242#134#0 649               0.0    819200.0  ...  6.111701e+06   
                    3496        2490368.0         0.0  ...  3.366213e+09   
                    3495        3145728.0         0.0  ...  4.412756e+09   
                    3503        3276800.0         0.0  ...  4.649491e+09   
                    3524        3014656.0         0.0  ...  4.390411e+09   

                                        iops  intensity  \
proc_name           time_range                            
app#corona242#134#0 649             7.460572   0.000001   
                    3496        25682.170802   0.000008   
                    3495        33666.654181   0.000008   
                    3503        35472.801083   0.000008   
                    3524        33496.177778   0.000008   

                                                                     id  \
proc_name           time_range                                            
app#corona242#134#0 649         326121582806776426917330803125765567622   
                    3496        214910685410521407681150529119621955637   
                    3495        316122203013112279036770411208111137187   
                    3503        143504805840061581447312501680945153396   
                    3524        289611916922526767567310878159036510115   

                                    att_perf  time_norm  time_pero  time_perr  \
proc_name           time_range                                                  
app#corona242#134#0 649             7.460572   0.788613   0.788613   0.019077   
                    3496        25682.170802   0.004353   0.004353   0.000105   
                    3495        33666.654181   0.004194   0.004194   0.000101   
                    3503        35472.801083   0.004146   0.004146   0.000100   
                    3524        33496.177778   0.004040   0.004040   0.000098   

                                time_score  time_threshold  
proc_name           time_range                              
app#corona242#134#0 649          very high           0.750  
                    3496           trivial           0.001  
                    3495           trivial           0.001  
                    3503           trivial           0.001  
                    3524           trivial           0.001  

[5 rows x 47 columns]

time: 184 ms (started: 2023-11-21 17:00:01 -08:00)


In [38]:
bot_time_pn_ll.loc['app#corona242#134#0'].sort_values('time_pero', ascending=False).head()

time  \
proc_name           time_range file_name                                                      
app#corona242#134#0 649        /p/lustre1/haridev/dlio_profiler_data/file_0-16...  0.134038   
                    3496       /p/lustre1/haridev/dlio_profiler_data/file_134-...  0.000740   
                    3495       /p/lustre1/haridev/dlio_profiler_data/file_134-...  0.000713   
                    3503       /p/lustre1/haridev/dlio_profiler_data/file_134-...  0.000705   
                    3524       /p/lustre1/haridev/dlio_profiler_data/file_134-...  0.000687   

                                                                                   count  \
proc_name           time_range file_name                                                   
app#corona242#134#0 649        /p/lustre1/haridev/dlio_profiler_data/file_0-16...      1   
                    3496       /p/lustre1/haridev/dlio_profiler_data/file_134-...     19   
                    3495       /p/lustre1/haridev/dlio_profiler_data/file_134-...     24   
                    3503       /p/lustre1/haridev/dlio_profiler_data/file_134-...     25   
                    3524       /p/lustre1/haridev/dlio_profiler_data/file_134-...     23   

                                                                                   size_min  \
proc_name           time_range file_name                                                      
app#corona242#134#0 649        /p/lustre1/haridev/dlio_profiler_data/file_0-16...    819200   
                    3496       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   
                    3495       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   
                    3503       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   
                    3524       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   

                                                                                   size_max  \
proc_name           time_range file_name                                                      
app#corona242#134#0 649        /p/lustre1/haridev/dlio_profiler_data/file_0-16...    819200   
                    3496       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   
                    3495       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   
                    3503       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   
                    3524       /p/lustre1/haridev/dlio_profiler_data/file_134-...    131072   

                                                                                      size  \
proc_name           time_range file_name                                                     
app#corona242#134#0 649        /p/lustre1/haridev/dlio_profiler_data/file_0-16...   819200   
                    3496       /p/lustre1/haridev/dlio_profiler_data/file_134-...  2490368   
                    3495       /p/lustre1/haridev/dlio_profiler_data/file_134-...  3145728   
                    3503       /p/lustre1/haridev/dlio_profiler_data/file_134-...  3276800   
                    3524       /p/lustre1/haridev/dlio_profiler_data/file_134-...  3014656   

                                                                                   read_time  \
proc_name           time_range file_name                                                       
app#corona242#134#0 649        /p/lustre1/haridev/dlio_profiler_data/file_0-16...   0.000000   
                    3496       /p/lustre1/haridev/dlio_profiler_data/file_134-...   0.000740   
                    3495       /p/lustre1/haridev/dlio_profiler_data/file_134-...   0.000713   
                    3503       /p/lustre1/haridev/dlio_profiler_data/file_134-...   0.000705   
                    3524       /p/lustre1/haridev/dlio_profiler_data/file_134-...   0.000687   

                                                                                   write_time  \
proc_name      

time: 175 ms (started: 2023-11-21 17:00:02 -08:00)


In [39]:
bot_time_fl_hl['time'] = bot_time_fl_hl['time'].astype(float)
bot_time_fl_hl['size'] = bot_time_fl_hl['size'].astype(int)
bot_time_fl_hl['count'] = bot_time_fl_hl['count'].astype(int)
bot_time_fl_hl_sw = bot_time_fl_hl.query('time_threshold > 0.01 & (write_time / time) > 0.75 & (size / count) < 262144')
bot_time_fl_hl_sw.compute()

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
file_name,,,,,,,,,,,,,,,,,,,,,
/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat,0.140953,1160,131072,819200,262144000,0.029744,0.140953,0.0,131072000.0,131072000.0,...,1.859796e+09,8229.689851,0.000004,3055255802399697147857971694728649660791372449...,1.779778e+06,1.0,1.0,0.048488,critical,0.9


time: 248 ms (started: 2023-11-21 17:00:02 -08:00)


In [40]:
res.main_view.reset_index()['proc_name'].nunique().compute()

160

time: 71.6 ms (started: 2023-11-21 17:00:02 -08:00)


In [41]:
bot_time_pn_hl['time'] = bot_time_pn_hl['time'].astype(float)
bot_time_pn_hl['size'] = bot_time_pn_hl['size'].astype(int)
bot_time_pn_hl['count'] = bot_time_pn_hl['count'].astype(int)
bot_time_pn_hl.query('time_threshold > 0.01 & (write_time / time) > 0.75 & (size / count) < 262144').compute()

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
proc_name,,,,,,,,,,,,,,,,,,,,,
app#corona239#23#0,0.123416,1001,131072,819200,131891200,0.026429,0.096987,0.0,131072000.0,819200.0,...,1.068670e+09,8110.767619,0.000008,1154505415543991293291412772764548002676268251...,2.004399e+06,0.726120,0.726120,0.017566,high,0.50
app#corona239#30#0,0.136576,1001,131072,819200,131891200,0.029514,0.107062,0.0,131072000.0,819200.0,...,9.656970e+08,7329.243149,0.000008,6453361618307755121778390408753534844427707753...,1.795913e+06,0.803547,0.803547,0.019439,very high,0.75
app#corona239#36#0,0.140793,1001,131072,819200,131891200,0.029892,0.110901,0.0,131072000.0,819200.0,...,9.367749e+08,7109.736209,0.000008,3175322929908849778290654389400062638091483723...,1.777655e+06,0.828355,0.828355,0.020039,very high,0.75
app#corona240#40#0,0.114953,1001,131072,819200,131891200,0.022787,0.092166,0.0,131072000.0,819200.0,...,1.147351e+09,8707.921659,0.000008,1476687908571057957277334629960923007661654916...,2.325832e+06,0.676325,0.676325,0.016361,high,0.50
app#corona240#42#0,0.122993,1001,131072,819200,131891200,0.021725,0.101268,0.0,131072000.0,819200.0,...,1.072343e+09,8138.643508,0.000008,1730854970236405147611990875067941085829784319...,2.447516e+06,0.723633,0.723633,0.017505,high,0.50
app#corona240#43#0,0.121777,1001,131072,819200,131891200,0.025617,0.096160,0.0,131072000.0,819200.0,...,1.083055e+09,8219.939119,0.000008,3310221501400504188318787636252308624344853253...,2.072564e+06,0.716476,0.716476,0.017332,high,0.50
app#corona240#44#0,0.133886,1001,131072,819200,131891200,0.025819,0.108067,0.0,131072000.0,819200.0,...,9.851018e+08,7476.517660,0.000008,1165464052140669177093835636031005652553072172...,2.057539e+06,0.787718,0.787718,0.019056,very high,0.75
app#corona240#45#0,0.138570,1001,131072,819200,131891200,0.027826,0.110744,0.0,131072000.0,819200.0,...,9.517999e+08,7223.769720,0.000008,3720169179922551384462031433842638071123531728...,1.921656e+06,0.815279,0.815279,0.019722,very high,0.75
app#corona240#46#0,0.149611,1001,131072,819200,131891200,0.026615,0.122997,0.0,131072000.0,819200.0,...,8.815581e+08,6690.663352,0.000008,3257858968333064407362012566600829087515262985...,2.002229e+06,0.880240,0.880240,0.021294,very high,0.75


time: 285 ms (started: 2023-11-21 17:00:02 -08:00)


In [42]:
bot_time_fl_hl_sw.eval('(write_time / time) > 0.75').head()

file_name
/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat    True
dtype: bool

time: 174 ms (started: 2023-11-21 17:00:03 -08:00)


In [43]:
bot_time_tr_hl.sort_values('time_pero', ascending=False).head()

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
time_range,,,,,,,,,,,,,,,,,,,,,
652,0.140953,1,819200,819200,819200,0.0,0.140953,0.0,0.0,819200.0,...,5.811864e+06,7.094560,0.000001,180482028116617673613735651520577555088,7.094560,1.000000,1.000000,0.041349,critical,0.90
650,0.136297,1,819200,819200,819200,0.0,0.136297,0.0,0.0,819200.0,...,6.010415e+06,7.336932,0.000001,186689416568516306385118971722209054298,7.336932,0.966965,0.966965,0.039983,critical,0.90
649,0.134038,1,819200,819200,819200,0.0,0.134038,0.0,0.0,819200.0,...,6.111701e+06,7.460572,0.000001,326121582806776426917330803125765567622,7.460572,0.950940,0.950940,0.039321,critical,0.90
648,0.132231,1,819200,819200,819200,0.0,0.132231,0.0,0.0,819200.0,...,6.195230e+06,7.562537,0.000001,271233160550087725884116749957151786118,7.562537,0.938119,0.938119,0.038791,critical,0.90
645,0.126266,2,819200,819200,1638400,0.0,0.126266,0.0,0.0,1638400.0,...,1.297573e+07,15.839517,0.000001,3146234945541156382688027267126553478662977243...,15.839532,0.895805,0.895805,0.037041,very high,0.75


time: 192 ms (started: 2023-11-21 17:00:03 -08:00)


In [44]:
bot_time_tr_hl.sort_values('time_pero', ascending=False).head().eval('size / count')

TypeError: Cannot interpret 'uint64[pyarrow]' as a data type

time: 1.29 s (started: 2023-11-21 17:00:03 -08:00)


In [ ]:
bot_time_tr_hl['time'] = bot_time_tr_hl['time'].astype(float)
bot_time_tr_hl['size'] = bot_time_tr_hl['size'].astype(int)
bot_time_tr_hl['count'] = bot_time_tr_hl['count'].astype(int)
bot_time_tr_hl.query('time_threshold > 0.01 & (write_time / time) > 0.75 & (size / count) < 262144').compute()

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
time_range,,,,,,,,,,,,,,,,,,,,,


time: 241 ms (started: 2023-11-19 19:37:24 -08:00)


In [ ]:
bot_time_fl_ml = res.bottlenecks['time'][('file_name',)].mid_level_view
bot_time_fl_ml.compute()

time  \
file_name                                          proc_name                      
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0   0.103754   
                                                   app#corona239#1#0   0.056857   
                                                   app#corona239#10#0  0.074339   
                                                   app#corona239#11#0  0.063459   
                                                   app#corona239#12#0  0.070210   
...                                                                         ...   
/p/lustre1/haridev/dlio_profiler_data/file_95-1... app#corona241#95#0  0.029323   
/p/lustre1/haridev/dlio_profiler_data/file_96-1... app#corona241#96#0  0.030245   
/p/lustre1/haridev/dlio_profiler_data/file_97-1... app#corona241#97#0  0.029729   
/p/lustre1/haridev/dlio_profiler_data/file_98-1... app#corona241#98#0  0.029460   
/p/lustre1/haridev/dlio_profiler_data/file_99-1... app#corona241#99#0  0.030642   

                                                                       count  \
file_name                                          proc_name                   
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0    1001   
                                                   app#corona239#1#0       1   
                                                   app#corona239#10#0      1   
                                                   app#corona239#11#0      1   
                                                   app#corona239#12#0      1   
...                                                                      ...   
/p/lustre1/haridev/dlio_profiler_data/file_95-1... app#corona241#95#0   1000   
/p/lustre1/haridev/dlio_profiler_data/file_96-1... app#corona241#96#0   1000   
/p/lustre1/haridev/dlio_profiler_data/file_97-1... app#corona241#97#0   1000   
/p/lustre1/haridev/dlio_profiler_data/file_98-1... app#corona241#98#0   1000   
/p/lustre1/haridev/dlio_profiler_data/file_99-1... app#corona241#99#0   1000   

                                                                       size_min  \
file_name                                          proc_name                      
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0     131072   
                                                   app#corona239#1#0     819200   
                                                   app#corona239#10#0    819200   
                                                   app#corona239#11#0    819200   
                                                   app#corona239#12#0    819200   
...                                                                         ...   
/p/lustre1/haridev/dlio_profiler_data/file_95-1... app#corona241#95#0    131072   
/p/lustre1/haridev/dlio_profiler_data/file_96-1... app#corona241#96#0    131072   
/p/lustre1/haridev/dlio_profiler_data/file_97-1... app#corona241#97#0    131072   
/p/lustre1/haridev/dlio_profiler_data/file_98-1... app#corona241#98#0    131072   
/p/lustre1/haridev/dlio_profiler_data/file_99-1... app#corona241#99#0    131072   

                                                                       size_max  \
file_name                                          proc_name                      
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0     819200   
                                                   app#corona239#1#0     819200   
                                                   app#corona239#10#0    819200   
                                                   app#corona239#11#0    819200   
                                                   app#corona239#12#0    819200   
...                                                                         ...   
/p/lustre1/haridev/dlio_profiler_data/file_95-1... app#corona241#95#0    131072   
/p/lustre1/haridev/dlio_profiler_data/file_96-1... app#corona241#96#0    131072   
/p/lustre1/haridev/dlio_profiler_data/file_9

time: 250 ms (started: 2023-11-17 15:47:25 -08:00)


In [ ]:
bot_time_fl_ml.loc['/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat'].compute()

time  \
file_name                                          proc_name                       
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0    0.103754   
                                                   app#corona239#1#0    0.056857   
                                                   app#corona239#10#0   0.074339   
                                                   app#corona239#11#0   0.063459   
                                                   app#corona239#12#0   0.070210   
...                                                                          ...   
                                                   app#corona242#155#0  0.072903   
                                                   app#corona242#156#0  0.063199   
                                                   app#corona242#157#0  0.070884   
                                                   app#corona242#158#0  0.066551   
                                                   app#corona242#159#0  0.073145   

                                                                        count  \
file_name                                          proc_name                    
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0     1001   
                                                   app#corona239#1#0        1   
                                                   app#corona239#10#0       1   
                                                   app#corona239#11#0       1   
                                                   app#corona239#12#0       1   
...                                                                       ...   
                                                   app#corona242#155#0      1   
                                                   app#corona242#156#0      1   
                                                   app#corona242#157#0      1   
                                                   app#corona242#158#0      1   
                                                   app#corona242#159#0      1   

                                                                        size_min  \
file_name                                          proc_name                       
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0      131072   
                                                   app#corona239#1#0      819200   
                                                   app#corona239#10#0     819200   
                                                   app#corona239#11#0     819200   
                                                   app#corona239#12#0     819200   
...                                                                          ...   
                                                   app#corona242#155#0    819200   
                                                   app#corona242#156#0    819200   
                                                   app#corona242#157#0    819200   
                                                   app#corona242#158#0    819200   
                                                   app#corona242#159#0    819200   

                                                                        size_max  \
file_name                                          proc_name                       
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0      819200   
                                                   app#corona239#1#0      819200   
                                                   app#corona239#10#0     819200   
                                                   app#corona239#11#0     819200   
                                                   app#corona239#12#0     819200   
...                                                                          ...   
                                                   app#corona242#155#0    819200   
                                                   app#corona242#156#0    819200

time: 241 ms (started: 2023-11-17 15:47:37 -08:00)


In [ ]:
bot_time_fl_ll = res.bottlenecks['time'][('file_name',)].low_level_view
bot_time_fl_ll.compute()

time  \
file_name                                          proc_name          time_range             
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0  619         0.074010   
                                                                      3358        0.000107   
                                                                      3477        0.000194   
                                                                      3478        0.000424   
                                                                      3479        0.000369   
...                                                                                    ...   
/p/lustre1/haridev/dlio_profiler_data/file_99-1... app#corona241#99#0 3525        0.000748   
                                                                      3526        0.000716   
                                                                      3527        0.000642   
                                                                      3528        0.000790   
                                                                      3529        0.000439   

                                                                                  count  \
file_name                                          proc_name          time_range          
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0  619             1   
                                                                      3358            1   
                                                                      3477            5   
                                                                      3478           12   
                                                                      3479           10   
...                                                                                 ...   
/p/lustre1/haridev/dlio_profiler_data/file_99-1... app#corona241#99#0 3525           23   
                                                                      3526           19   
                                                                      3527           18   
                                                                      3528           22   
                                                                      3529           15   

                                                                                  size_min  \
file_name                                          proc_name          time_range             
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0  619           819200   
                                                                      3358          131072   
                                                                      3477          131072   
                                                                      3478          131072   
                                                                      3479          131072   
...                                                                                    ...   
/p/lustre1/haridev/dlio_profiler_data/file_99-1... app#corona241#99#0 3525          131072   
                                                                      3526          131072   
                                                                      3527          131072   
                                                                      3528          131072   
                                                                      3529          131072   

                                                                                  size_max  \
file_name                                          proc_name          time_range             
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0  619           819200   
                                                                      3358          131072   
                                                                      3477      

time: 247 ms (started: 2023-11-17 15:46:09 -08:00)


In [45]:
bot_time_fl_ll.loc['/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat'].compute()

time  \
file_name                                          proc_name           time_range             
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0   619         0.074010   
                                                                       3358        0.000107   
                                                                       3477        0.000194   
                                                                       3478        0.000424   
                                                                       3479        0.000369   
...                                                                                     ...   
                                                   app#corona242#155#0 618         0.072903   
                                                   app#corona242#156#0 613         0.063199   
                                                   app#corona242#157#0 617         0.070884   
                                                   app#corona242#158#0 615         0.066551   
                                                   app#corona242#159#0 618         0.073145   

                                                                                   count  \
file_name                                          proc_name           time_range          
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0   619             1   
                                                                       3358            1   
                                                                       3477            5   
                                                                       3478           12   
                                                                       3479           10   
...                                                                                  ...   
                                                   app#corona242#155#0 618             1   
                                                   app#corona242#156#0 613             1   
                                                   app#corona242#157#0 617             1   
                                                   app#corona242#158#0 615             1   
                                                   app#corona242#159#0 618             1   

                                                                                   size_min  \
file_name                                          proc_name           time_range             
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0   619           819200   
                                                                       3358          131072   
                                                                       3477          131072   
                                                                       3478          131072   
                                                                       3479          131072   
...                                                                                     ...   
                                                   app#corona242#155#0 618           819200   
                                                   app#corona242#156#0 613           819200   
                                                   app#corona242#157#0 617           819200   
                                                   app#corona242#158#0 615           819200   
                                                   app#corona242#159#0 618           819200   

                                                                                   size_max  \
file_name                                          proc_name           time_range             
/p/lustre1/haridev/dlio_profiler_data/file_0-16... app#corona239#0#0   619           819200   
                                                                       3358          131072   
                                      

time: 195 ms (started: 2023-11-21 17:05:25 -08:00)


In [ ]:
bot_time_fl_hl2 = dd.read_json('hmm/*.jsonl', orient='records', lines=True)
bot_time_fl_hl2

,file_name,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,metadata_size,read_count,write_count,metadata_count,read_min,read_max,write_min,write_max,metadata_min,metadata_max,data_count,data_size,data_time,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,close_time,open_time,seek_time,stat_time,close_count,open_count,seek_count,stat_count,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,string,float64,int64,int64,int64,int64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,int64,float64,float64,float64,float64,string,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 115 ms (started: 2023-11-09 21:22:55 -08:00)


In [ ]:
jsonbag = dask.bag.read_text('hmm/*.jsonl')
jsonbag

dask.bag<bag-from-delayed, npartitions=1>

time: 3.99 ms (started: 2023-11-09 21:29:01 -08:00)


In [ ]:
import json
import jmespath
import mistql
from datetime import datetime

def x(y):
    record = json.loads(y)
    t0 = datetime.now()
    result = mistql.query('(metadata_time / time) > 0.5', record)
    print(result, (datetime.now() - t0))
    return y

jsonbag.map(x).compute()

False 0:00:00.038827
False 0:00:00.037421
False 0:00:00.037500
False 0:00:00.037539
False 0:00:00.036638
False 0:00:00.036557
False 0:00:00.037311
False 0:00:00.036504
False 0:00:00.037200
False 0:00:00.037465
False 0:00:00.037242
False 0:00:00.037339
False 0:00:00.037053
False 0:00:00.038028
False 0:00:00.037267
False 0:00:00.036873
False 0:00:00.038833
False 0:00:00.036582
False 0:00:00.036293
False 0:00:00.037136
False 0:00:00.037871
False 0:00:00.037195
False 0:00:00.037011
False 0:00:00.037250
False 0:00:00.038066
False 0:00:00.036631
False 0:00:00.036972
False 0:00:00.037296
False 0:00:00.037956
False 0:00:00.037726
False 0:00:00.036674
False 0:00:00.036560
False 0:00:00.037878
False 0:00:00.036668
False 0:00:00.036829
False 0:00:00.036637
False 0:00:00.037748
False 0:00:00.036333
False 0:00:00.036650
False 0:00:00.036952
False 0:00:00.037519
False 0:00:00.036679
False 0:00:00.036910
False 0:00:00.036353
False 0:00:00.036954
False 0:00:00.035996
False 0:00:00.036714
False 0:00:00

['{"file_name":"\\/p\\/lustre1\\/haridev\\/dlio_profiler_data\\/file_0-160.bat","time":0.140953064,"count":1160,"size_min":131072,"size_max":819200,"size":262144000,"read_time":0.0297439098,"write_time":0.140953064,"metadata_time":0.0,"read_size":131072000.0,"write_size":131072000.0,"metadata_size":0.0,"read_count":1000.0,"write_count":160.0,"metadata_count":0.0,"read_min":7077888.0,"read_max":7077888.0,"write_min":131072000.0,"write_max":131072000.0,"metadata_min":0.0,"metadata_max":0.0,"data_count":1160.0,"data_size":262144000.0,"data_time":0.140953064,"sequential_time":0.140953064,"random_time":0.0,"sequential_size":262144000,"random_size":0,"sequential_count":1160,"random_count":0,"close_time":0.0,"open_time":0.0,"seek_time":0.0,"stat_time":0.0,"close_count":0.0,"open_count":0.0,"seek_count":0.0,"stat_count":0.0,"bw":1859796393.3964817524,"iops":8229.6898511502,"intensity":0.000004425,"id":-2256903632684378025,"att_perf":1779777.8944064551,"time_norm":1.0,"time_pero":1.0,"time_perr

time: 3.59 s (started: 2023-11-09 21:47:09 -08:00)


In [ ]:
bot_time_fl_hl2.head()

,file_name,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,...,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
0,/p/lustre1/haridev/dlio_profiler_data/file_0-1...,0.140953,1160,131072,819200,262144000,0.029744,0.140953,0,131072000,...,1.859796e+09,8229.689851,0.000004,-2256903632684378025,1.779778e+06,1.000000,1.000000,0.048488,critical,0.9
1,/p/lustre1/haridev/dlio_profiler_data/file_10-...,0.028973,1000,131072,131072,131072000,0.028973,0.000000,0,131072000,...,4.523957e+09,34515.055011,0.000008,-7683100460611653235,1.831552e+06,0.205550,0.205550,0.009967,low,0.1
2,/p/lustre1/haridev/dlio_profiler_data/file_108...,0.029569,1000,131072,131072,131072000,0.029569,0.000000,0,131072000,...,4.432692e+09,33818.759424,0.000008,-4876108725053092184,1.807881e+06,0.209782,0.209782,0.010172,low,0.1
3,/p/lustre1/haridev/dlio_profiler_data/file_109...,0.030145,1000,131072,131072,131072000,0.030145,0.000000,0,131072000,...,4.348061e+09,33173.074337,0.000008,1773396103887177671,1.769308e+06,0.213865,0.213865,0.010370,low,0.1
4,/p/lustre1/haridev/dlio_profiler_data/file_11-...,0.029704,1000,131072,131072,131072000,0.029704,0.000000,0,131072000,...,4.412555e+09,33665.122924,0.000008,-4509324401508286302,1.813493e+06,0.210739,0.210739,0.010218,low,0.1


time: 85.5 ms (started: 2023-11-09 21:23:05 -08:00)


In [ ]:
bot_time_fl_hl['time_score'].value_counts().compute()

time_score
low         93
critical     1
Name: count, dtype: int64[pyarrow]

time: 159 ms (started: 2023-11-09 21:15:37 -08:00)


In [ ]:
res.bottlenecks['time'][('file_name',)].high_level_view.compute()

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,bw,iops,intensity,id,att_perf,time_norm,time_pero,time_perr,time_score,time_threshold
file_name,,,,,,,,,,,,,,,,,,,,,
/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat,0.140953,1160,131072,819200,262144000,0.029744,0.140953,0.0,131072000.0,131072000.0,...,1.859796e+09,8229.689851,0.000004,-2256903632684378025,1.779778e+06,1.000000,1.000000,0.048488,critical,0.9
/p/lustre1/haridev/dlio_profiler_data/file_10-160.bat,0.028973,1000,131072,131072,131072000,0.028973,0.000000,0.0,131072000.0,0.0,...,4.523957e+09,34515.055011,0.000008,-7683100460611653235,1.831552e+06,0.205550,0.205550,0.009967,low,0.1
/p/lustre1/haridev/dlio_profiler_data/file_108-160.bat,0.029569,1000,131072,131072,131072000,0.029569,0.000000,0.0,131072000.0,0.0,...,4.432692e+09,33818.759424,0.000008,-4876108725053092184,1.807881e+06,0.209782,0.209782,0.010172,low,0.1
/p/lustre1/haridev/dlio_profiler_data/file_109-160.bat,0.030145,1000,131072,131072,131072000,0.030145,0.000000,0.0,131072000.0,0.0,...,4.348061e+09,33173.074337,0.000008,1773396103887177671,1.769308e+06,0.213865,0.213865,0.010370,low,0.1
/p/lustre1/haridev/dlio_profiler_data/file_11-160.bat,0.029704,1000,131072,131072,131072000,0.029704,0.000000,0.0,131072000.0,0.0,...,4.412555e+09,33665.122924,0.000008,-4509324401508286302,1.813493e+06,0.210739,0.210739,0.010218,low,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/p/lustre1/haridev/dlio_profiler_data/file_95-160.bat,0.029323,1000,131072,131072,131072000,0.029323,0.000000,0.0,131072000.0,0.0,...,4.469959e+09,34103.082389,0.000008,2804043938870366739,1.815851e+06,0.208033,0.208033,0.010087,low,0.1
/p/lustre1/haridev/dlio_profiler_data/file_96-160.bat,0.030245,1000,131072,131072,131072000,0.030245,0.000000,0.0,131072000.0,0.0,...,4.333666e+09,33063.244441,0.000008,-7650266009452005792,1.761128e+06,0.214575,0.214575,0.010404,low,0.1
/p/lustre1/haridev/dlio_profiler_data/file_97-160.bat,0.029729,1000,131072,131072,131072000,0.029729,0.000000,0.0,131072000.0,0.0,...,4.408839e+09,33636.774825,0.000008,2132469358832555709,1.794322e+06,0.210917,0.210917,0.010227,low,0.1


time: 270 ms (started: 2023-11-09 21:14:49 -08:00)


In [ ]:
res.view_results['time'][('file_name',)].view.head()

,,,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,seek_time,stat_time,close_count,open_count,seek_count,stat_count,bw,iops,intensity,id
time_range,proc_name,file_name,,,,,,,,,,,,,,,,,,,,,
619,corona239#app#0#0,/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat,0.074010,1,819200,819200,819200,0.000000,0.07401,0.0,0.0,819200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.106879e+07,13.511707,0.000001,-9190021740228276738
3358,corona239#app#0#0,/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat,0.000107,1,131072,131072,131072,0.000107,0.00000,0.0,131072.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.229879e+09,9383.230425,0.000008,-1751412563354664547
3477,corona239#app#0#0,/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat,0.000194,5,131072,131072,655360,0.000194,0.00000,0.0,655360.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.385196e+09,25826.995074,0.000008,-5719599644426800361
3478,corona239#app#0#0,/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat,0.000424,12,131072,131072,1572864,0.000424,0.00000,0.0,1572864.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.710388e+09,28308.013498,0.000008,788041111675198064
3479,corona239#app#0#0,/p/lustre1/haridev/dlio_profiler_data/file_0-160.bat,0.000369,10,131072,131072,1310720,0.000369,0.00000,0.0,1310720.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.553690e+09,27112.501616,0.000008,2897324484792828361


time: 50.5 ms (started: 2023-11-09 21:10:08 -08:00)
